In [22]:
import connectAndQuery as cq
import  pandas as pd
stocks = cq.get_db('localhost:27017', 'dan1', 'movieDetails')
res = stocks.find({'runtime':144}).limit(110)
l = list(res)
df = pd.DataFrame(l)


In [23]:
df.head(5)

,_id,title,year,rated,runtime,countries,genres,director,writers,actors,plot,poster,imdb,tomato,metacritic,awards,type
0,5cb9c05f432e41200cdea5f1,Guilty of Romance,2011.0,None,144.0,[Japan],"[Drama, Horror]",Shion Sono,[Shion Sono],"[Miki Mizuno, Makoto Togashi, Megumi Kagurazak...","A grisly murder occurs in Maruyama-cho, Shibuy...",http://ia.media-imdb.com/images/M/MV5BMTUwNjMz...,"{'id': 'tt1743724', 'rating': 6.8, 'votes': 25...","{'meter': 69.0, 'image': 'fresh', 'rating': 5....",56.0,"{'wins': 1.0, 'nominations': 1.0, 'text': '1 w...",movie
1,5cb9c060432e41200cdeaaaf,Oh! What a Lovely War,1969.0,G,144.0,[UK],"[Comedy, Musical, War]",Richard Attenborough,"[Charles Chilton, Ted Allan]","[Wendy Allnutt, Colin Farrell, Malcolm McFee, ...",A movie about the First World War based on a s...,http://ia.media-imdb.com/images/M/MV5BMTMxNzEz...,"{'id': 'tt0064754', 'rating': 7.2, 'votes': 19...",NaN,NaN,"{'wins': 7.0, 'nominations': 7.0, 'text': 'Won...",movie
2,5cb9c060432e41200cdeadfe,The Martian,2015.0,PG-13,144.0,"[USA, UK]","[Adventure, Drama, Sci-Fi]",Ridley Scott,"[Drew Goddard, Andy Weir]","[Matt Damon, Jessica Chastain, Kristen Wiig, J...","During a manned mission to Mars, Astronaut Mar...",NaN,"{'id': 'tt3659388', 'rating': 8.2, 'votes': 18...","{'meter': 93.0, 'image': 'certified', 'rating'...",80.0,NaN,movie


In [63]:
import json
#Load pipeline structure
f = open('samplePipeline.json')
data = json.load(f)
# Closing file
f.close()


querySample = pd.read_csv('queryTemplates.csv')
mask1 = querySample['name'] == data[1]['query']
query = querySample[mask1]['template'][0]


In [68]:
newQuery = query.replace('var1', "['Shion Sono','Richard Attenborough']")
eval(newQuery)

{'director': {'$in': ['Shion Sono', 'Richard Attenborough']}}

In [56]:
pipeInstructions = pd.read_json('samplePipeline.json')
mask = pipeInstructions['id'] == 2
# pipeInstructions[mask]['query'][0]
pipeInstructions[mask]['query'][1]

{'director': {'$in': []}}

In [48]:
def runQuery(df, id):
    import connectAndQuery as cq
    import  pandas as pd
    mask = df['id'] == id
    query = df[mask]['query'][0]
    cursor = cq.get_db('localhost:27017', 'dan1', 'movieDetails')
    res = cursor.find(query).limit(10)
    l = list(res)
    df = pd.DataFrame(l)
    return df

In [49]:
pipeInstructions.pipe(runQuery,1)

,_id,title,year,rated,runtime,countries,genres,director,writers,actors,plot,poster,imdb,tomato,metacritic,awards,type
0,5cb9c05f432e41200cdea5f1,Guilty of Romance,2011.0,None,144.0,[Japan],"[Drama, Horror]",Shion Sono,[Shion Sono],"[Miki Mizuno, Makoto Togashi, Megumi Kagurazak...","A grisly murder occurs in Maruyama-cho, Shibuy...",http://ia.media-imdb.com/images/M/MV5BMTUwNjMz...,"{'id': 'tt1743724', 'rating': 6.8, 'votes': 25...","{'meter': 69.0, 'image': 'fresh', 'rating': 5....",56.0,"{'wins': 1.0, 'nominations': 1.0, 'text': '1 w...",movie
1,5cb9c060432e41200cdeaaaf,Oh! What a Lovely War,1969.0,G,144.0,[UK],"[Comedy, Musical, War]",Richard Attenborough,"[Charles Chilton, Ted Allan]","[Wendy Allnutt, Colin Farrell, Malcolm McFee, ...",A movie about the First World War based on a s...,http://ia.media-imdb.com/images/M/MV5BMTMxNzEz...,"{'id': 'tt0064754', 'rating': 7.2, 'votes': 19...",NaN,NaN,"{'wins': 7.0, 'nominations': 7.0, 'text': 'Won...",movie
2,5cb9c060432e41200cdeadfe,The Martian,2015.0,PG-13,144.0,"[USA, UK]","[Adventure, Drama, Sci-Fi]",Ridley Scott,"[Drew Goddard, Andy Weir]","[Matt Damon, Jessica Chastain, Kristen Wiig, J...","During a manned mission to Mars, Astronaut Mar...",NaN,"{'id': 'tt3659388', 'rating': 8.2, 'votes': 18...","{'meter': 93.0, 'image': 'certified', 'rating'...",80.0,NaN,movie


In [34]:
relevantDataList = []
for i in range(len(data)):
    dfPerStep = pd.DataFrame()
    if data[i]['type'] == 'MongoQuery':
        dfPerStep = runQuery(data[i]['query'])
        tempDf = dfPerStep.copy()
        relevantDataList.append(tempDf)
        
data = relevantDataList[0]

In [39]:
def increaseRuntime(df, n):
    df['runtime'] += n
    df.to_excel('tryoutExport.xlsx')
    return df

In [41]:
data.pipe(increaseRuntime, 5)

,_id,title,year,rated,runtime,countries,genres,director,writers,actors,plot,poster,imdb,tomato,metacritic,awards,type
0,5cb9c05f432e41200cdea5f1,Guilty of Romance,2011.0,None,164.0,[Japan],"[Drama, Horror]",Shion Sono,[Shion Sono],"[Miki Mizuno, Makoto Togashi, Megumi Kagurazak...","A grisly murder occurs in Maruyama-cho, Shibuy...",http://ia.media-imdb.com/images/M/MV5BMTUwNjMz...,"{'id': 'tt1743724', 'rating': 6.8, 'votes': 25...","{'meter': 69.0, 'image': 'fresh', 'rating': 5....",56.0,"{'wins': 1.0, 'nominations': 1.0, 'text': '1 w...",movie
1,5cb9c060432e41200cdeaaaf,Oh! What a Lovely War,1969.0,G,164.0,[UK],"[Comedy, Musical, War]",Richard Attenborough,"[Charles Chilton, Ted Allan]","[Wendy Allnutt, Colin Farrell, Malcolm McFee, ...",A movie about the First World War based on a s...,http://ia.media-imdb.com/images/M/MV5BMTMxNzEz...,"{'id': 'tt0064754', 'rating': 7.2, 'votes': 19...",NaN,NaN,"{'wins': 7.0, 'nominations': 7.0, 'text': 'Won...",movie
2,5cb9c060432e41200cdeadfe,The Martian,2015.0,PG-13,164.0,"[USA, UK]","[Adventure, Drama, Sci-Fi]",Ridley Scott,"[Drew Goddard, Andy Weir]","[Matt Damon, Jessica Chastain, Kristen Wiig, J...","During a manned mission to Mars, Astronaut Mar...",NaN,"{'id': 'tt3659388', 'rating': 8.2, 'votes': 18...","{'meter': 93.0, 'image': 'certified', 'rating'...",80.0,NaN,movie
